In [1]:
!pip install kaggle

# https://www.kaggle.com/datasets/najamahmed97/audio-recording-whisper/data

In [2]:
!kaggle datasets download -d najamahmed97/audio-recording-whisper

Dataset URL: https://www.kaggle.com/datasets/najamahmed97/audio-recording-whisper
License(s): Community Data License Agreement - Permissive - Version 1.0
 99% 972M/986M [00:13<00:00, 75.9MB/s]
100% 986M/986M [00:13<00:00, 75.2MB/s]


In [3]:
!unzip audio-recording-whisper.zip

Archive:  audio-recording-whisper.zip
  inflating: audio_recordings/Audio_Recordings/CAR0001.mp3  
  inflating: audio_recordings/Audio_Recordings/CAR0002.mp3  
  inflating: audio_recordings/Audio_Recordings/CAR0003.mp3  
  inflating: audio_recordings/Audio_Recordings/CAR0004.mp3  
  inflating: audio_recordings/Audio_Recordings/CAR0005.mp3  
  inflating: audio_recordings/Audio_Recordings/DER0001.mp3  
  inflating: audio_recordings/Audio_Recordings/GAS0001.mp3  
  inflating: audio_recordings/Audio_Recordings/GAS0002.mp3  
  inflating: audio_recordings/Audio_Recordings/GAS0003.mp3  
  inflating: audio_recordings/Audio_Recordings/GAS0004.mp3  
  inflating: audio_recordings/Audio_Recordings/GAS0005.mp3  
  inflating: audio_recordings/Audio_Recordings/GAS0007.mp3  
  inflating: audio_recordings/Audio_Recordings/GEN0001.mp3  
  inflating: audio_recordings/Audio_Recordings/MSK0001.mp3  
  inflating: audio_recordings/Audio_Recordings/MSK0003.mp3  
  inflating: audio_recordings/Audio_Recordings/

In [4]:
!pip install pydub

In [5]:
from pydub import AudioSegment
import os

def convert_mp3_to_wav(mp3_folder, wav_folder):
    """Convert all mp3 files in a folder to wav format."""
    if not os.path.exists(wav_folder):
        os.makedirs(wav_folder)

    for filename in os.listdir(mp3_folder):
        if filename.endswith('.mp3'):
            mp3_path = os.path.join(mp3_folder, filename)
            wav_path = os.path.join(wav_folder, filename.replace('.mp3', '.wav'))

            # Load and convert the mp3 file to wav
            audio = AudioSegment.from_mp3(mp3_path)
            audio.export(wav_path, format="wav")

# Example usage
mp3_folder = 'audio_recordings/Audio_Recordings'
wav_folder = 'wav'
convert_mp3_to_wav(mp3_folder, wav_folder)


In [6]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 15.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803321 sha256=06b2988f8cc78b7fe190235c0ced2d602083993c7fd87fce0ae86b1f5c198662
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [7]:
import whisper
import os

def transcribe_audio_files(wav_folder, transcription_folder):
    """Transcribe WAV files in a folder using Whisper model."""
    if not os.path.exists(transcription_folder):
        os.makedirs(transcription_folder)

    # Load Whisper model
    model = whisper.load_model("base")

    for filename in os.listdir(wav_folder):
        if filename.endswith('.wav'):
            wav_path = os.path.join(wav_folder, filename)
            result = model.transcribe(wav_path)

            # Save the transcription as a text file
            transcription_file = os.path.join(transcription_folder, filename.replace('.wav', '.txt'))
            with open(transcription_file, 'w') as f:
                f.write(result['text'])

# Example usage
transcription_folder = 'path/to/save/transcriptions'
transcribe_audio_files(wav_folder, transcription_folder)

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 121MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [8]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.1 MB/s eta 0:00:00


In [14]:
import jiwer
import os

def calculate_kpis(transcription_folder, reference_transcripts_folder):
    """Calculate accuracy, precision, recall, f1 score, deletion rate, insertion rate."""

    kpi_results = {
        "accuracy": [],
        "precision": [],
        "recall": [],
        "f1_score": [],
        "deletion_rate": [],
        "insertion_rate": []
    }

    for filename in os.listdir(transcription_folder):
        if filename.endswith('.txt'):
            transcription_file = os.path.join(transcription_folder, filename)
            reference_file = os.path.join(reference_transcripts_folder, filename)

            # Read the transcription and reference text
            with open(transcription_file, 'r', encoding='utf-8') as f: # Keep utf-8 for transcriptions
                transcription = f.read()

            # Try reading with 'utf-8' encoding first, then fallback to 'utf-16' and finally 'latin-1'
            try:
                with open(reference_file, 'r', encoding='utf-8') as f: # Try utf-8 first
                    reference = f.read()
            except UnicodeDecodeError:
                try:
                    with open(reference_file, 'r', encoding='utf-16') as f: # Fallback to utf-16
                        reference = f.read()
                except UnicodeDecodeError:
                    with open(reference_file, 'r', encoding='latin-1') as f: # Final fallback to latin-1
                        reference = f.read()

            # Calculate WER, precision, recall, and F1 using jiwer with transformations
            wer = jiwer.wer(reference, transcription,
                            truth_transform=jiwer.transformations.wer_default,
                            hypothesis_transform=jiwer.transformations.wer_default)

            measures = jiwer.compute_measures(reference, transcription)

            kpi_results["accuracy"].append(1 - wer)  # 1 - Word Error Rate (WER)
            kpi_results["precision"].append(measures['wip']) # Word Information Preserved
            kpi_results["recall"].append(measures['wil']) # Word Information Lost
            kpi_results["f1_score"].append(measures['mer']) # Match Error Rate
            # Avoiding division by zero errors by adding a small value to the denominator
            kpi_results["deletion_rate"].append(measures['deletions'] / (measures['substitutions'] + 1e-10))
            kpi_results["insertion_rate"].append(measures['insertions'] / (measures['substitutions'] + 1e-10))

    # Aggregate KPIs (calculate average)
    avg_kpis = {k: sum(v)/len(v) for k, v in kpi_results.items()}

    return avg_kpis

# Example usage
reference_transcripts_folder = 'audio_recordings/Clean_Transcripts'
kpi_results = calculate_kpis(transcription_folder, reference_transcripts_folder)

print("KPI Results:", flush=True)
for kpi, value in kpi_results.items():
    print(f"{kpi.capitalize()}: {value:.4f}", flush=True)

KPI Results:
Accuracy: 0.6779
Precision: 0.5381
Recall: 0.4619
F1_score: 0.3144
Deletion_rate: 0.5822
Insertion_rate: 0.1160
